In [21]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
Alpha_Name = "AlphaNet_Original_Input_12.14"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "5d_ret"

In [3]:
data = pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_5d_return/Final/2016-01-01_2016-03-01.par")
data

,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
0,2016-01-04 09:30:00,1,493.686822,100.297978,233.622631,848.107538,118.870156,215.930329,462.890987,48.074000,...,0.409564,0.063916,-0.397551,0.798538,0.403656,0.552324,1.583324,0.800347,1.095105,0.622714
1,2016-01-05 09:30:00,1,432.396820,49.027262,340.551782,735.228142,74.149009,384.986453,352.920684,24.332840,...,-0.070611,0.359575,-1.388393,0.792393,0.485904,0.522871,1.571133,0.963431,1.036695,0.595218
2,2016-01-06 09:30:00,1,337.971018,109.962562,959.914513,584.777067,78.857796,1056.321373,248.357680,69.037551,...,0.031607,0.758193,-1.141885,0.727955,0.603935,0.515858,1.443384,1.197476,1.022789,0.778602
3,2016-01-07 09:30:00,1,303.798249,285.786027,1219.175082,517.489902,270.479938,1205.942453,209.043831,226.646311,...,-0.214489,0.587138,-0.769965,0.682133,0.618636,0.495635,1.352536,1.226624,0.982700,0.950525
4,2016-01-08 09:30:00,1,309.432873,447.018827,1307.530622,478.841693,413.633627,1404.571589,191.728973,342.114307,...,-0.127053,0.588696,-1.568135,0.627447,0.682844,0.425711,1.244100,1.353944,0.844053,0.407136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100415,2016-02-23 09:30:00,603999,82.546333,13.220744,3.939839,64.556478,13.388256,4.094640,63.597444,11.013922,...,-4.019131,-0.788358,1.434060,33.695005,20.521491,16.546807,33.695005,20.521491,16.546807,-1.553833
100416,2016-02-24 09:30:00,603999,97.501361,8.654742,4.598367,73.989994,8.152866,5.015200,75.610106,5.526532,...,-2.878182,-0.053333,1.378724,31.666902,19.551269,16.623331,31.666902,19.551269,16.623331,-1.961587
100417,2016-02-25 09:30:00,603999,89.556589,3.085533,4.362713,69.703233,3.318556,4.575711,63.003850,2.035990,...,-4.065793,-0.239076,0.154095,30.533060,18.110360,16.718509,30.533060,18.110360,16.718509,-1.105204
100418,2016-02-26 09:30:00,603999,53.090480,2.417344,4.463398,41.824933,2.722233,4.601460,39.290122,1.961300,...,-4.592209,0.990322,-1.782736,29.053336,18.296711,16.472976,29.053336,18.296711,16.472976,-1.629470


In [42]:
import DataAPI
import datetime
def convert_to_standard_daily_data_par(df: pd.DataFrame, output_name: str,  output_path: str):
    grouped = df.groupby('timestamp')
    for date, group in grouped:
        date_format = pd.to_datetime(date).date()
        assert DataAPI.is_trading_day(date), f"{date} is not a trading date!"
        file_name = datetime.date.strftime(date_format, '%Y%m%d') + '.par'
        folder = os.path.join(output_path, output_name, str(date_format.year))
        if not os.path.exists(folder):
            os.makedirs(folder)
        file = os.path.join(folder, file_name)
        group.to_parquet(file)
    return None
def convert_to_standard_daily_data_csv(df: pd.DataFrame, output_name: str,  output_path: str):
    grouped = df.groupby('timestamp')
    for date, group in grouped:
        date_format = pd.to_datetime(date).date()
        assert DataAPI.is_trading_day(date), f"{date} is not a trading date!"
        file_name = datetime.date.strftime(date_format, '%Y%m%d') + '.csv'
        folder = os.path.join(output_path, output_name, str(date_format.year))
        if not os.path.exists(folder):
            os.makedirs(folder)
        file = os.path.join(folder, file_name)
        group.to_csv(file, sep=',', mode='w', header=False, index=False, encoding='utf-8')
    return None
class DataLoader(object):
    def __init__(self, model, device=None):
        self.feature_data = None
        self.target = None
        self.length = None
        self.y = None
    def load_data_from_file(data_path,end_date,start_date = "2015-01-01"):
        time_list  = DataAPI.get_trading_days(start_date,end_date)
        final = []
        for date in time_list:
            date = data_path + str(date.year) + '/' + str(date.year) + str(date.month).zfill(2) + str(date.day).zfill(2) + ".par"
            if (os.path.exists(date)):
                final.append(pd.read_parquet(date))
        self.feature_data = pd.concat(final,axis=0)
        self.length = self.feature_data.shape[0]
        return self.feature_data
    
    def to_torch_DataLoader(sequence,shuffle,batch_size = 1024,num_workers = 16):
        self.target = pd.DataFrame(self.feature_data["target"])
        self.feature_data.drop("target",axis=1,inplace=True)
        self.feture_data.set_index(["timestamp","ticker"],inplace=True)
        x = torch.from_numpy(np.array(self.feture_data).reshape(self.length,-1,sequence))
        y = torch.from_numpy(np.array(self.target).reshape(-1,1))
        self.feature_data = torch.utils.data.TensorDataset(x, y)
        self.feature_data = torch.utils.data.DataLoader(
            dataset=self.feature_data,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            pin_memory=True
        )
        return self.feature_data

In [ ]:
importrtrtport shui

In [17]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/feature_platform")
alpha_list = list(FT.features_in_path.keys())[:]
alpha_list = ['ewmCorr(high,low,7)','ewmCov(open,high,7)']
print(alpha_list)
start_date = '2015-01-01'
end_date = '2022-01-01'
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   3%|███                                                                                      | 116/3410 [00:00<00:03, 844.87it/s]

['ewmCorr(high,low,7)', 'ewmCov(open,high,7)']


Loading Feature...: 100%|████████████████████████████████████████████████████████████████████████████████████████| 3410/3410 [00:08<00:00, 404.84it/s]


(                            ewmCorr(high,low,7)  ewmCov(open,high,7)
 timestamp           ticker                                          
 2016-02-02 09:30:00 000638             1.175631            -0.032220
                     300365             1.175631            -0.031230
                     600462             1.175631            -0.032169
                     002075             1.175631            -0.032206
                     002435             1.175631            -0.032302
 ...                                         ...                  ...
 2021-07-01 09:30:00 300230            -4.442220            -0.037525
                     002319            -4.504204            -0.036942
                     603286            -4.533666            -0.037445
                     000796            -4.536809            -0.033297
                     002707            -4.536859            -0.020746
 
 [4444115 rows x 2 columns],
 None)

Process ForkPoolWorker-534:
Process ForkPoolWorker-517:
Process ForkPoolWorker-567:
Process ForkPoolWorker-556:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._

In [19]:
data = FT.feature_data

In [ ]:
convert_to_standard_daily_data_csv

In [41]:
output_path = r'/home/wuwenjun/feature_platform/ti0/wuwenjun'
for i in data.columns[:1]:
    one_data = pd.DataFrame(data[i]).unstack()
    display(one_data)
    mean = one_data.rolling(30).mean()
    std = one_data.rolling(30).std()
    one_data = ((one_data- mean) /std ).stack()
    one_data.columns = one_data.columns + "_time_series_norm"
    display(one_data)
    convert_to_standard_daily_data_csv(one_data, one_data.columns.to_list()[0], output_path)

In [40]:
one_data.columns.to_list()[0]

'ewmCorr(high,low,7)_time_series_norm'

In [15]:
"ewmCov(open,high,8)" in alpha_list

False

In [36]:
output_path = r'/home/wuwenjun/Data_lib/ti0/wuwenjun'
convert_to_standard_daily_data_csv(data,"a",output_path)

In [64]:
start_date = '2015-01-01'
end_date = '2020-03-01'
load_data_from_file(data_path = "/home/wuwenjun/Data_lib/ti0/wuwenjun/a/",start_date = start_date,end_date = end_date)

,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
0,2015-02-16 09:30:00,1,1047.509016,165.116922,249.851660,924.464829,243.390500,183.194222,926.891342,158.088300,...,0.138453,-0.376716,0.284385,1.522075,1.245018,0.876160,3.017927,2.468587,1.737222,-0.679514
5,2015-02-16 09:30:00,2,5734.296871,3871.305252,2296.206307,4441.200376,4079.822467,1564.848673,4471.782349,3782.803471,...,-0.248100,0.016542,0.237585,2.459175,2.413624,1.709360,2.963325,2.906449,2.058258,-0.230876
10,2015-02-16 09:30:00,4,1.642506,5.157167,1.294367,1.901233,6.506182,1.342306,0.990144,5.159844,...,-0.025775,0.338164,0.477255,2.458642,3.225611,1.455813,3.328256,4.366485,1.970716,-0.066228
15,2015-02-16 09:30:00,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.758071
20,2015-02-16 09:30:00,6,72.269658,54.692880,11.679193,56.316516,53.318840,8.058444,54.038229,46.167377,...,-0.552169,0.418084,0.274916,1.837805,2.273738,0.967687,3.028318,3.855613,1.640955,-0.120982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12914,2015-02-27 09:30:00,603806,0.990602,0.123747,0.357178,1.009622,0.105296,0.297494,1.039184,0.037416,...,0.951249,-0.318816,0.397691,4.102738,2.095189,1.601320,4.102738,2.095189,1.601320,0.234638
12919,2015-02-27 09:30:00,603889,1.434967,0.388298,0.158693,1.513833,0.312260,0.127571,1.436000,0.261711,...,1.065205,-0.562555,0.248467,20.095335,6.400258,3.862024,20.095335,6.400258,3.862024,-0.520455
12924,2015-02-27 09:30:00,603988,4.081573,1.347522,0.415591,3.966544,1.351102,0.430010,3.653089,1.108484,...,2.216744,-0.733896,0.315015,10.635233,4.137585,2.203580,10.635233,4.137585,2.203580,-0.017941
12929,2015-02-27 09:30:00,603993,0.128444,0.110044,0.057189,0.114267,0.084144,0.068788,0.117044,0.029156,...,2.337364,0.303475,1.850804,1.678938,1.607478,0.980598,13.672502,13.090613,7.985736,0.021513


In [58]:

for date in a:
    date = data_path + str(date.year) + '/' + str(date.year) + str(date.month).zfill(2) + str(date.day).zfill(2) + ".par"
    if (os.path.exists(date)):
        display(pd.read_parquet(date))

,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
0,2015-02-16 09:30:00,1,1047.509016,165.116922,249.851660,924.464829,243.390500,183.194222,926.891342,158.088300,...,0.138453,-0.376716,0.284385,1.522075,1.245018,0.876160,3.017927,2.468587,1.737222,-0.679514
5,2015-02-16 09:30:00,2,5734.296871,3871.305252,2296.206307,4441.200376,4079.822467,1564.848673,4471.782349,3782.803471,...,-0.248100,0.016542,0.237585,2.459175,2.413624,1.709360,2.963325,2.906449,2.058258,-0.230876
10,2015-02-16 09:30:00,4,1.642506,5.157167,1.294367,1.901233,6.506182,1.342306,0.990144,5.159844,...,-0.025775,0.338164,0.477255,2.458642,3.225611,1.455813,3.328256,4.366485,1.970716,-0.066228
15,2015-02-16 09:30:00,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.758071
20,2015-02-16 09:30:00,6,72.269658,54.692880,11.679193,56.316516,53.318840,8.058444,54.038229,46.167377,...,-0.552169,0.418084,0.274916,1.837805,2.273738,0.967687,3.028318,3.855613,1.640955,-0.120982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12910,2015-02-16 09:30:00,603806,0.443253,0.247522,0.639169,0.457222,0.524244,0.531124,0.403511,0.269633,...,-0.072316,-0.173693,0.066309,2.376789,3.073324,1.729418,2.376789,3.073324,1.729418,-0.468794
12915,2015-02-16 09:30:00,603889,5.337591,0.419260,0.359393,4.156316,0.671042,0.272120,3.629171,0.416376,...,-0.110045,-0.755051,0.222831,18.622211,10.945611,4.356080,18.622211,10.945611,4.356080,-0.651571
12920,2015-02-16 09:30:00,603988,0.801172,1.477298,0.696822,0.914261,3.249069,0.589089,0.546361,1.412584,...,-0.301102,-0.635924,0.259096,4.233925,7.557609,2.679651,4.233925,7.557609,2.679651,-0.600844
12925,2015-02-16 09:30:00,603993,0.167547,1.009791,0.165149,0.168873,1.040073,0.088347,0.093889,0.961456,...,0.059075,1.216520,0.401096,1.354129,2.014622,1.037911,11.027435,16.406229,8.452387,1.561955


,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
1,2015-02-17 09:30:00,1,1284.106922,304.746632,233.319300,1251.132811,325.347890,190.544389,1311.161194,268.878420,...,-1.631951,-0.601635,0.240629,1.601496,1.147820,0.839082,3.175407,2.275878,1.663689,-0.824485
6,2015-02-17 09:30:00,2,6188.724627,3409.274011,2128.461866,5480.067676,3678.223628,1526.930661,5534.657310,3015.082144,...,-2.024287,-0.278011,0.114105,2.595396,2.283293,1.598584,3.127465,2.749413,1.924882,-0.957525
11,2015-02-17 09:30:00,4,1.066050,2.880976,1.627113,1.298382,3.977423,1.723682,0.462517,2.433477,...,-0.423116,-0.007538,0.651345,2.427078,2.983371,1.483107,3.285515,4.038558,2.007671,0.362513
16,2015-02-17 09:30:00,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.634456
21,2015-02-17 09:30:00,6,59.732633,51.954194,11.928616,61.154788,48.484300,9.158411,62.935058,39.912906,...,-2.169973,0.153011,0.355811,2.072185,2.071422,0.966809,3.449016,3.512533,1.639447,-0.412160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12911,2015-02-17 09:30:00,603806,0.431211,0.116020,0.623369,0.473624,0.216796,0.501380,0.407393,0.040891,...,-0.053480,-0.420129,0.362262,2.790598,2.647962,1.747458,2.790598,2.647962,1.747458,-0.605057
12916,2015-02-17 09:30:00,603889,5.318951,0.601720,0.362637,4.308049,0.598762,0.285427,3.645447,0.467376,...,0.003582,-0.672707,0.255418,19.269685,9.160524,4.267842,19.269685,9.160524,4.267842,-0.580894
12921,2015-02-17 09:30:00,603988,0.456006,1.200691,0.727537,0.780172,1.939040,0.673266,0.349722,0.660544,...,0.341740,-0.942182,0.370875,5.330764,6.302500,2.578005,5.330764,6.302500,2.578005,-0.547468
12926,2015-02-17 09:30:00,603993,0.173567,0.778507,0.166596,0.148547,0.801982,0.095498,0.143249,0.700280,...,-1.430465,1.188729,0.690102,1.300769,1.829196,1.001487,10.592887,14.896169,8.155824,0.798530


,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
2,2015-02-25 09:30:00,1,1515.538580,320.840611,242.242430,1522.025569,325.437271,214.401980,1539.154187,259.118036,...,-1.377793,-0.151133,0.292173,1.574869,1.078356,0.797058,3.122595,2.138158,1.580369,-0.790990
7,2015-02-25 09:30:00,2,6105.016367,2194.069159,1961.410916,5765.322744,2771.184769,1590.713092,5585.910544,1806.598854,...,-1.402413,-0.275325,0.332445,2.657700,2.363516,1.479860,3.202540,2.845969,1.781915,-1.009345
12,2015-02-25 09:30:00,4,0.987011,0.807691,2.274988,0.911471,0.931042,2.430443,0.383169,0.748182,...,0.516496,0.109667,0.496909,2.633889,2.672365,1.621002,3.565484,3.617547,2.194335,0.370950
17,2015-02-25 09:30:00,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.551774
22,2015-02-25 09:30:00,6,81.445376,28.686078,12.873478,80.499390,27.341320,10.946467,76.078407,20.230664,...,-1.253071,0.016998,0.547058,2.006838,1.922735,0.979178,3.354449,3.260413,1.660418,-0.480094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12912,2015-02-25 09:30:00,603806,0.467339,0.174700,0.639036,0.468403,0.157911,0.543153,0.441606,0.103600,...,0.914222,-0.049958,0.286271,3.655478,2.428542,1.724236,3.655478,2.428542,1.724236,-0.489249
12917,2015-02-25 09:30:00,603889,4.720442,0.779107,0.303300,3.238298,0.616937,0.256733,2.717656,0.627874,...,1.415511,-0.311116,0.247464,20.922393,7.968798,3.996335,20.922393,7.968798,3.996335,-0.485382
12922,2015-02-25 09:30:00,603988,0.550406,1.498967,0.741344,0.786756,1.381427,0.705883,0.474200,1.096298,...,2.161298,-0.313384,0.270556,7.838995,5.548322,2.506327,7.838995,5.548322,2.506327,-0.536367
12927,2015-02-25 09:30:00,603993,0.192967,0.404711,0.177977,0.159350,0.521150,0.117800,0.156256,0.371883,...,-0.911940,0.807058,0.425784,1.164484,1.667491,0.939749,9.483038,13.579300,7.653064,1.210065


,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
3,2015-02-26 09:30:00,1,1466.712161,355.378729,244.332118,1546.573639,318.650776,222.730596,1381.349806,297.507667,...,-0.375218,-0.482991,0.038896,1.643798,1.006125,0.739729,3.259280,1.994944,1.466684,-1.006040
8,2015-02-26 09:30:00,2,4974.296989,1913.226433,1751.026071,5037.266604,1814.848254,1347.104296,3929.405933,1739.757034,...,-0.112862,-0.594442,-0.170422,2.846118,2.204524,1.449747,3.428993,2.654531,1.745642,-1.099272
13,2015-02-26 09:30:00,4,1.717798,0.538161,2.732580,1.537449,0.521843,2.801038,1.194233,0.598796,...,1.040980,0.313818,0.718845,3.040787,2.489409,1.859869,4.116295,3.369878,2.517698,0.079871
18,2015-02-26 09:30:00,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.750926
23,2015-02-26 09:30:00,6,79.733978,16.906731,14.098111,77.113129,15.771430,12.803842,67.512751,11.226143,...,-0.472540,-0.346255,0.467476,2.054685,1.726778,0.958582,3.448120,2.928136,1.625493,-0.642333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12913,2015-02-26 09:30:00,603806,0.698847,0.153233,0.508400,0.713083,0.135167,0.406806,0.697134,0.080828,...,0.848249,-0.136787,0.059387,3.991075,2.251771,1.666884,3.991075,2.251771,1.666884,-0.377750
12918,2015-02-26 09:30:00,603889,1.583540,0.460847,0.206197,1.382298,0.421833,0.173214,1.338882,0.374991,...,1.150775,-0.537218,-0.103647,21.157687,7.188015,3.916033,21.157687,7.188015,3.916033,-0.462839
12923,2015-02-26 09:30:00,603988,2.788588,1.335356,0.595781,2.851211,1.313424,0.547990,2.291566,1.083118,...,1.779462,-0.492095,0.106791,9.423998,4.653593,2.363616,9.423998,4.653593,2.363616,-0.573730
12928,2015-02-26 09:30:00,603993,0.140942,0.143724,0.174602,0.146404,0.222997,0.120718,0.114344,0.104727,...,0.431771,1.270116,-0.071522,1.352609,1.667138,0.874760,11.015042,13.576453,7.123780,0.894336


,timestamp,ticker,0,1,2,3,4,5,6,7,...,315,316,317,318,319,320,321,322,323,target
4,2015-02-27 09:30:00,1,1271.480107,335.406489,121.012389,1338.104279,252.973056,186.957378,1296.812044,215.993378,...,-0.411738,-0.287113,0.381687,1.581575,1.159289,0.769491,3.135918,2.298631,1.525691,-1.213373
9,2015-02-27 09:30:00,2,3007.011124,1966.513336,534.521598,3576.475804,1794.562296,720.097829,2894.788109,1703.299238,...,0.765047,-0.705638,0.543562,2.955702,2.361851,1.466831,3.560564,2.843945,1.766209,-1.424857
14,2015-02-27 09:30:00,4,3.005956,0.685427,3.632744,3.253250,0.731718,3.774500,2.665078,0.731380,...,1.033364,0.407242,0.786864,3.082118,2.479404,1.984749,4.172242,3.356344,2.686742,-0.161959
19,2015-02-27 09:30:00,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.648621
24,2015-02-27 09:30:00,6,53.426327,11.189049,13.840252,54.834750,11.517256,15.121572,45.357113,6.597123,...,-0.170960,-0.458285,0.760655,1.985062,1.641400,1.020253,3.340571,2.783365,1.730085,-0.661651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12914,2015-02-27 09:30:00,603806,0.990602,0.123747,0.357178,1.009622,0.105296,0.297494,1.039184,0.037416,...,0.951249,-0.318816,0.397691,4.102738,2.095189,1.601320,4.102738,2.095189,1.601320,0.234638
12919,2015-02-27 09:30:00,603889,1.434967,0.388298,0.158693,1.513833,0.312260,0.127571,1.436000,0.261711,...,1.065205,-0.562555,0.248467,20.095335,6.400258,3.862024,20.095335,6.400258,3.862024,-0.520455
12924,2015-02-27 09:30:00,603988,4.081573,1.347522,0.415591,3.966544,1.351102,0.430010,3.653089,1.108484,...,2.216744,-0.733896,0.315015,10.635233,4.137585,2.203580,10.635233,4.137585,2.203580,-0.017941
12929,2015-02-27 09:30:00,603993,0.128444,0.110044,0.057189,0.114267,0.084144,0.068788,0.117044,0.029156,...,2.337364,0.303475,1.850804,1.678938,1.607478,0.980598,13.672502,13.090613,7.985736,0.021513


In [65]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np

config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(FT.features_in_path.keys())[:]

alpha_list = ["LSTM_AlphaNet_Original_Input_5d_return_1","AlphaNet_Original_Input_5d_return_5d_change_1",
              "LSTM_AlphaNet_Original_Input_5d_return_2","AlphaNet_Original_Input_5d_return_5d_change_2",
             "LSTM_AlphaNet_Original_Input_5d_return_3","AlphaNet_Original_Input_5d_return_5d_change_3",
              "LSTM_AlphaNet_Original_Input_5d_return_4","AlphaNet_Original_Input_5d_return_5d_change_4",
              "LSTM_AlphaNet_Original_Input_5d_return_5","AlphaNet_Original_Input_5d_return_5d_change_5"
             ]


print(alpha_list)
start_date = '2020-01-01'
end_date = '2021-06-01'
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   3%|███                                                                                      | 116/3400 [00:00<00:05, 601.19it/s]

['LSTM_AlphaNet_Original_Input_5d_return_1', 'AlphaNet_Original_Input_5d_return_5d_change_1', 'LSTM_AlphaNet_Original_Input_5d_return_2', 'AlphaNet_Original_Input_5d_return_5d_change_2', 'LSTM_AlphaNet_Original_Input_5d_return_3', 'AlphaNet_Original_Input_5d_return_5d_change_3', 'LSTM_AlphaNet_Original_Input_5d_return_4', 'AlphaNet_Original_Input_5d_return_5d_change_4', 'LSTM_AlphaNet_Original_Input_5d_return_5', 'AlphaNet_Original_Input_5d_return_5d_change_5']


Loading Feature...:  87%|████████████████████████████████████████████████████████████████████████████           | 2971/3400 [00:02<00:00, 1190.07it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(                            LSTM_AlphaNet_Original_Input_5d_return_1  \
 timestamp           ticker                                             
 2020-01-09 09:30:00 300081                                  9.835197   
                     603069                                  7.704015   
                     603133                                  7.525295   
                     600580                                  5.281538   
                     600583                                  4.635966   
 ...                                                              ...   
 2021-05-31 09:30:00 600613                                       NaN   
                     300554                                       NaN   
                     601005                                       NaN   
                     600439                                       NaN   
                     600257                                       NaN   
 
                             AlphaNet_Original_In

In [66]:
FT.feature_data

LSTM_AlphaNet_Original_Input_5d_return_1  \
timestamp           ticker                                             
2020-01-09 09:30:00 300081                                  9.835197   
                    603069                                  7.704015   
                    603133                                  7.525295   
                    600580                                  5.281538   
                    600583                                  4.635966   
...                                                              ...   
2021-05-31 09:30:00 600613                                       NaN   
                    300554                                       NaN   
                    601005                                       NaN   
                    600439                                       NaN   
                    600257                                       NaN   

                            AlphaNet_Original_Input_5d_return_5d_change_1  \
timestamp           ticker                                                  
2020-01-09 09:30:00 300081                                      -0.382247   
                    603069                                       0.168119   
                    603133                                       0.361857   
                    600580                                      -0.261308   
                    600583                                       4.080823   
...                                                                   ...   
2021-05-31 09:30:00 600613                                            NaN   
                    300554                                            NaN   
                    601005                                            NaN   
                    600439                                            NaN   
                    600257                                            NaN   

                            LSTM_AlphaNet_Original_Input_5d_return_2  \
timestamp           ticker                                             
2020-01-09 09:30:00 300081                                       NaN   
                    603069                                       NaN   
                    603133                                       NaN   
                    600580                                       NaN   
                    600583                                       NaN   
...                                                              ...   
2021-05-31 09:30:00 600613                                       NaN   
                    300554                                       NaN   
                    601005                                       NaN   
                    600439                                       NaN   
                    600257                                       NaN   

                            AlphaNet_Original_Input_5d_return_5d_change_2  \
timestamp           ticker                                                  
2020-01-09 09:30:00 300081                                            NaN   
                    603069                                            NaN   
                    603133                                            NaN   
                    600580                                            NaN   
                    600583                                            NaN   
...                                                                   ...   
2021-05-31 09:30:00 600613                                            NaN   
                    300554                                            NaN   
                    601005                                            NaN   
                    600439                                            NaN   
                    600257                                            NaN   

                            LSTM_AlphaNet_Original_Input_5d_return_3  \
timestamp           ticker                                             
2020-0

In [17]:
import Research.utils.namespace as namespace
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)

FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/Data_lib")
alpha_list = list(FT.features_in_path.keys())[-1:]
print(alpha_list)
start_date = '2015-02-01'
end_date = '2015-03-01'
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
# FT.load_return_data()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:  40%|█████████████████████████████████████▌                                                        | 6/15 [00:00<00:00, 95.00it/s]

['a']


Loading Feature...:  67%|██████████████████████████████████████████████████████████████                               | 10/15 [00:02<00:01,  4.66it/s]


ValueError: y cannot be empty!

In [ ]:
   def load_feature_from_file(self, feature_names, start_time, end_time, universe='Investable', timedelta=None,
                               transformer=None, cover_rate=False, check_timestamp=False):
        # init
        self.reset()
        assert self.pathway is not None, \
            "if you want to upload from file, please assign initial parameter `feature_path`!"
        missing_file_allowed = self.options.daily_data_missing_allowed
        if timedelta is not None:
            self.time_delta = timedelta
        if isinstance(feature_names, str):
            feature_names = [feature_names]

        # time range
        start_time = convert_to_datetime(start_time)
        end_time = convert_to_datetime(end_time)
        if end_time < \
                datetime.datetime.fromordinal(end_time.date().toordinal()) + datetime.timedelta(hours=9, minutes=30):
            trade_days = get_trading_days(start_time.date(), get_last_trading_day(end_time.date()), output='datetime')
        else:
            trade_days = get_trading_days(start_time.date(), end_time.date(), output='datetime')

        handler = getattr(Reader, f"{self.options.feature_data_format}_reader")
        self.trading_days = trade_days
        self.feature_names = feature_names

        # load files
        feature_path_list = []
        for feature_name in feature_names:
            feature_path_list.extend(
                [(os.path.join(self.pathway, self.features_in_path[feature_name],
                               feature_name, str(item.year), datetime.datetime.strftime(item, '%Y%m%d')
                               + '.' + self.options.feature_data_format), feature_name, item) for item in trade_days])

        cover_rate_value = None
        if universe == 'All' and cover_rate is False:
            reader = FeatureAnalysis.read_files
            self.feature_data = reader(feature_path_list, pool=self.pool,
                                       scope='Loading Feature...',
                                       handler=handler, time_delta=timedelta, transformer=transformer,
                                       missing_file_allowed=missing_file_allowed)
        else:
            reader = FeatureAnalysis.read_files_filter
            self.feature_data, cover_rate_value = reader(feature_path_list, universe=universe, pool=self.pool,
                                                         scope='Loading Feature...',
                                                         handler=handler, time_delta=timedelta, transformer=transformer,
                                                         cover_rate=cover_rate,
                                                         missing_file_allowed=missing_file_allowed)
        self.feature_data = \
            self.feature_data.loc[(self.feature_data.index.get_level_values(0) >= pd.Timestamp(start_time)) &
                                  (self.feature_data.index.get_level_values(0) <= pd.Timestamp(end_time))]

        if timedelta is not None:
            self.ti += self.time_delta

        # check timestamp
        if check_timestamp:
            time_list = self.feature_data.index.get_level_values(0).unique()
            time_list = list(set(zip(time_list.hour, time_list.minute)))
            assert len(time_list) == 1, f"Feature data generate at a different time, {time_list}"
            time_target = int(str(time_list[0][0]).zfill(2) + str(time_list[0][1]).zfill(2))
            ti = self.time_list.index(time_target)
            assert self.ti == ti, \
                f"Parameter `ti` in config file is different from `ti` in alpha files, {self.ti} vs {ti}"

        return self.feature_data, cover_rate_value

In [6]:
# os.mkdir(path+"Pooling_Norm_Daily")
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 176745.39it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [8]:
""" Multi Processing"""
pool = mp.Pool(10)
t1 = time.time()
def test(file):
    array = pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"])
    index = array.index
    array = np.array(array)
    array = torch.from_numpy(array.reshape(array.shape[0],-1,3))

    # Pooling
    max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
    avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
    min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
    array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
    array = array.flatten(start_dim=1)
    array = pd.DataFrame(array.detach().numpy())
    array.index = index
    array.reset_index(inplace=True)
    # Daily Norm
    final = []
    for day in tqdm(array['timestamp'].drop_duplicates()):
        df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
        index = df.index
        # Batch
        df = torch.from_numpy(np.array(df)).to(torch.float) 
        batch = nn.BatchNorm1d(df.shape[1])
        df = batch(df)
        df = pd.DataFrame(df.detach().numpy())
        df.index = index
        final.append(df)

    final = pd.concat(final,axis=0)
    final.columns = [str(i) for i in range(final.shape[1])]
    final.reset_index(inplace=True)

    final.to_parquet(path + "Pooling_Norm_Daily/" + file)
    print("Save to : ", path + "Pooling_Norm_Daily/" + file)
    return None
for file in path_list:
#     test(file)
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 5/5 [00:00<00:00, 61.07it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2015-01-01_2015-03-01.par


 45%|████▌     | 30/66 [00:05<00:06,  5.45it/s]

Save to : 

 59%|█████▉    | 38/64 [00:06<00:04,  5.47it/s]

...

/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-01-01_2020-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2018-03-01_2018-06-01.par


 27%|██▋       | 17/62 [00:00<00:00, 57.50it/s]

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2018-06-01_2018-09-01.par


100%|██████████| 62/62 [00:01<00:00, 61.85it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2019-03-01_2019-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2019-06-01_2019-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-03-01_2020-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2021-01-01_2021-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2018-09-01_2019-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-06-01_2020-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2019-09-01_2020-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2020-09-01_2021-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2021-03-01_2021-06-01.par
并行执行时间： 42


In [10]:
pd.set_option('display.max_columns', None)
pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Pooling_Norm_Daily/2015-01-01_2015-03-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,2015-02-16 09:30:00,1,0.207835,0.021603,-0.232302,0.102923,0.104148,0.236395,0.278850,-0.225329,-0.100755,0.259385,0.421858,0.381663,0.444228,0.814598,0.173874,0.571642,1.175144,1.073323,0.444840,0.815275,0.174882,0.444565,0.815076,0.175516,-0.248215,-0.279964,-0.047211,-0.297900,-0.999507,-1.038000,0.133856,0.223983,0.229365,0.505350,1.091140,1.245504,-0.589995,-0.407401,0.176291,0.506039,1.092819,1.246501,0.505738,1.092584,1.247309,0.029127,-0.119623,0.195165,0.142636,0.223226,0.337103,0.481127,0.325459,0.231147,-0.456628,0.138279,0.554142,0.482520,0.327595,0.231049,0.482168,0.326304,0.230653,0.114225,-0.168267,-0.433872,0.233194,-0.510114,-0.251352,0.816742,0.798614,0.119052,0.234074,-0.507536,-0.249942,0.233896,-0.510094,-0.251845,0.529395,0.698129,0.578325,-0.807911,-0.176545,0.453865,0.530590,0.700219,0.579743,0.530259,0.699260,0.579030,-1.783878,-1.379385,-0.789560,0.227190,0.327937,0.364581,0.227202,0.326820,0.362730,-1.781165,-1.378666,-0.788926,-1.784895,-1.381820,-0.790502,0.227200,0.325990,0.361133,0.244314,0.283043,0.413693,0.245476,0.274015,0.390765,0.203350,0.250616,0.370272,0.231255,0.246264,0.255954,0.231790,0.266179,0.404866,1.623192,1.870501,2.521021,0.301921,-0.086809,-0.238868,-0.592634,-0.586870,-0.324868,-0.569037,-0.552629,-0.141187,-0.264807,-0.243747,-0.175149,-0.267160,-0.245432,-0.177168,-0.264807,-0.243451,-0.175149,-0.270408,-0.247713,-0.179367,-0.263521,-0.240379,-0.168633,0.220825,0.490968,0.877159,-0.038199,-0.067795,-1.161185,0.223819,0.494465,0.881653,0.225341,0.495021,0.881904,-0.815820,-0.929328,-0.545952,-0.668898,-0.960868,-0.449766,-1.017549,-1.135066,-0.549489,-0.848561,-0.943469,-0.614387,-0.778471,-0.952729,-0.458424,-0.150269,-0.150425,-0.050158,-0.299700,-0.036359,0.249617,-0.049063,-0.051807,-0.035366,-0.050169,-0.052982,-0.037844
1,2015-02-16 09:30:00,2,-0.008425,0.686842,0.923933,0.032671,0.302827,0.353685,-0.078937,0.582956,0.785258,0.059245,0.479820,0.740284,0.633762,0.666464,-0.190145,0.082877,0.658991,0.716015,0.633893,0.663907,-0.196824,0.634039,0.662821,-0.197979,0.231052,0.574803,0.620721,0.075810,0.410627,0.426701,0.186548,0.355746,0.433584,0.384214,0.023905,-0.521522,-0.232829,0.089149,-0.231224,0.384982,0.020998,-0.531177,0.384642,0.018376,-0.535036,0.094713,0.273760,0.354074,0.244432,0.385729,0.483805,0.778020,0.260817,-0.321160,-0.276150,0.370434,0.620650,0.778888,0.259054,-0.329360,0.779255,0.256923,-0.331461,0.249710,0.302733,0.213305,0.670804,-0.086282,-0.396093,-0.316182,0.048106,0.159994,0.671138,-0.090386,-0.408804,0.671525,-0.093790,-0.413617,0.591612,0.075723,-0.377689,0.023809,0.350679,0.018272,0.592768,0.072884,-0.387845,0.592534,0.070210,-0.391467,0.096950,-0.865420,-1.170692,0.227190,0.327914,0.364529,0.227202,0.326787,0.362655,0.097425,-0.870091,-1.169711,0.096430,-0.873500,-1.171344,0.227200,0.325989,0.361131,0.615152,0.958852,1.345552,0.701523,0.952714,1.513892,0.556350,0.844346,1.257248,0.646635,0.891480,1.006592,0.651538,0.935476,1.410979,4.162855,4.644673,5.654445,0.922344,0.863064,0.124509,-0.190290,-0.065724,0.418306,-0.380133,-0.301098,0.164518,-0.264807,-0.243747,-0.175149,-0.267160,-0.245432,-0.177168,-0.264807,-0.243451,-0.175149,-0.270408,-0.247713,-0.179367,-0.263521,-0.240379,-0.168633,0.070268,0.043035,0.395784,-0.042480,-0.058868,-0.467706,0.068629,0.044270,0.399711,0.068741,0.043565,0.